# EDE - Interplanetary Probes - Radar Calculation

## Best Case: Arecibo-to-Echo 2-to-Arecibo

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const

In [2]:
def snr(p_r, delta_p_n):
    return p_r / delta_p_n

In [3]:
def p_r(p_t, g_t, g_r, lamda, sigma, r):
    return (p_t * g_t * g_r * lamda**2 * sigma) / ((4 * np.pi)**3 * r**4)

In [4]:
def delta_p_n(p_n, delta_f, delta_t):
    return p_n / np.sqrt(delta_f * delta_t)

Main EQ

In [5]:
def r(p_t, g_t, g_r, lamda, sigma, delta_f, delta_t, snr, p_n):
    return (((p_t * g_t * g_r * lamda**2 * sigma * np.sqrt(delta_f * delta_t))/((4 * np.pi)**3 * snr * p_n))**(1/4)).to(u.AU)

Sub-EQs

In [6]:
def get_p_n(t_sys, delta_f):
    return (const.k_B * t_sys * delta_f).to(u.W)

In [7]:
def get_delta_f(d, lamda, p_rot, frequency_resolution):
    #rotation_broadening = (d/(lamda * p_rot)).to(u.Hz)
    rotation_broadening = (4 * np.pi * d/(lamda * p_rot)).to(u.Hz)
    if rotation_broadening > frequency_resolution:
        return rotation_broadening
    else:
        return frequency_resolution


In [8]:
def get_g(epsilon, a, lamda):
    return (4 * np.pi * epsilon * a / lamda**2).to(u.W/u.W)

In [9]:
def get_sigma(alpha, d):
    return alpha * np.pi * d**2 / 4

*List of necessary constants*

In [10]:
diameter_of_echo2 = 41.1 * u.m #from Wikipedia
albedo_of_echo2 = 1 #assumed perfect
rotation_period_of_echo2 = 1 * u.day #assume tidal-locking

diameter_of_arecibo = 305 * u.m #from murray and jennet, also Busch
radius_of_arecibo = 0.5 * diameter_of_arecibo
area_of_arecibo = np.pi * (radius_of_arecibo)**2
efficiency_of_arecibo = 0.38 #from Busch
frequency_of_arecibo = 2380 * u.MHz #from murray and jennet, also Busch
transmitter_power_of_arecibo = 1 * u.MW #900 * u.kW from Busch
#murray and jennet
system_temperature_of_arecibo = 20 * u.K #23 * u.K from Busch
bandwidth_of_arecibo = 10 * u.MHz #from murray and jennet
frequency_resolution_of_arecibo = 1 * u.Hz #from taylor
time_resolution_of_arecibo = 1 * u.microsecond#from taylor, samples per baud??? I'm guessing here
integration_time_of_arecibo = 1 * u.hour

frequency_resolution = frequency_resolution_of_arecibo
#time_resolution = time_resolution_of_arecibo
integration_time = integration_time_of_arecibo

In [11]:
#don't have to worry about changing from transmit to receive (4 sec) b/c
#we'll be at much greater distances. Similarly, this won't set the time
#resolution

#radar albedo
alpha = albedo_of_echo2

#diameter of target
d = diameter_of_echo2

#aperture efficiency of transmitter
epsilon_t = efficiency_of_arecibo

#aperture efficiency of receiver
epsilon_r = efficiency_of_arecibo

#area of transmitter
a_t = area_of_arecibo

#area of receiver
a_r = area_of_arecibo

#wavelength of transmission
lamda = (const.c / frequency_of_arecibo).to(u.cm)

#transmitter power
p_t = transmitter_power_of_arecibo

#system temperature of receiver
t_sys = system_temperature_of_arecibo

#rotation period of target
p_rot = rotation_period_of_echo2

#time integration = match to pulse
#frequency integration = match to min capabilities

In [12]:
g_t = get_g(epsilon_t, a_t, lamda)
g_r = get_g(epsilon_r, a_r, lamda)
sigma = get_sigma(alpha, d)
delta_f = get_delta_f(d, lamda, p_rot, frequency_resolution)
delta_t = integration_time
snr = 5
p_n = get_p_n(t_sys, delta_f)

In [13]:
r(p_t, 
  g_t, 
  g_r, 
  lamda, 
  sigma, 
  delta_f, 
  delta_t, 
  snr, 
  p_n)

<Quantity 0.14524367 AU>